In [ ]:
import os
import sys
import pickle
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
import matplotlib.pyplot as plt

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

try:
    from pfcommon import *
except:
    sys.path.append('..')
    from pfcommon import *

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\Nine-bus System'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

In [ ]:
project = app.GetActiveProject()
if project is None:
    raise Exception('Cannot get active project')
print('Successfully obtained active project.')

In [ ]:
project_folders = {}
for folder_name in ('study',):
    project_folders[folder_name] = app.GetProjectFolder(folder_name)
    if project_folders[folder_name] is None:
        raise Exception(f'No folder "{folder_name}" present')
    print(f'Successfully obtained folder "{folder_name}".')

In [ ]:
sort_fun = lambda x: x.loc_name
generators = sorted(app.GetCalcRelevantObjects('*.ElmSym'), key=sort_fun)
lines = sorted(app.GetCalcRelevantObjects('*.ElmLne'), key=sort_fun)
buses = sorted(app.GetCalcRelevantObjects('*.ElmTerm'), key=sort_fun)
loads = sorted(app.GetCalcRelevantObjects('*.ElmLod'), key=sort_fun)
transformers = sorted(app.GetCalcRelevantObjects('*.ElmTr2'), key=sort_fun)
n_generators, n_lines, n_buses = len(generators), len(lines), len(buses)
n_loads, n_transformers = len(loads), len(transformers)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')
print(f'There are {n_transformers} transformers.')

In [ ]:
P = {}
S = {}
H = {}
for generator in generators:
    i = int(generator.loc_name[1:])
    generator_type = generator.GetAttribute('typ_id')
    P[i] = generator.GetAttribute('pgini')
    S[i] = generator_type.GetAttribute('sgn')
    H[i] = generator_type.GetAttribute('h')
    print(f'{generator.loc_name}: P = {P[i]:4.0f} MW, S = {S[i]:5.0f} MVA, inertia = {H[i]:5.2f} s.')

In [ ]:
for line in lines:
    line_type = line.GetAttribute('typ_id')
    vrating = line_type.GetAttribute('uline')
    print(f'{line.loc_name}: Vrating = {vrating:6.1f} kV.')

In [ ]:
for bus in buses:
    vrating = bus.GetAttribute('uknom')
    print(f'{bus.loc_name}: Vrating = {vrating:6.1f} kV.')

In [ ]:
for load in loads:
    plini = load.GetAttribute('plini')
    qlini = load.GetAttribute('qlini')
    print(f'{load.loc_name}: P = {plini:5.1f} MW, Q = {qlini:5.1f} MVAR.')

## Load flow analysis

In [ ]:
study_case_name = '01- Load Flow'
defaults = {
    'pgini':  [None, 163, 85],
    'plini': [125, 90, 100],
    'qlini': [50, 30, 35]
}
lf_res = {}
LAMBDA = np.r_[0 : 31] / 100
for coeff in LAMBDA:
    for generator, pgini in zip(generators, defaults['pgini']):
        if pgini is not None:
            generator.pgini = (1 + coeff) * pgini
    for load, plini, qlini in zip(loads, defaults['plini'], defaults['qlini']):
        load.plini = (1 + coeff) * plini
        load.qlini = (1 + coeff) * qlini
    lf_res[coeff] = run_load_flow(app, project_folders['study'], generators, loads, buses, study_case_name)
pickle.dump(lf_res, open('WSCC_9_bus_load_flow_PowerFactory_overload.pkl','wb'))

### Reset the parameters to their default values
for generator, pgini in zip(generators, defaults['pgini']):
    if pgini is not None:
        generator.pgini = pgini
for load, plini, qlini in zip(loads, defaults['plini'], defaults['qlini']):
    load.plini = plini
    load.qlini = qlini

In [ ]:
study_case_name = '01- Load Flow'
lf_res = run_load_flow(app, project_folders['study'], study_case_name,
                       generators, loads, buses, lines, transformers)
pickle.dump(lf_res, open('WSCC_9_bus_load_flow_PowerFactory_1_km_lines.pkl','wb'))
print_load_flow(lf_res)